In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy as np
import pickle
import gc
import re
# ここから自作
import model
import result
import mode
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
# 時刻を表示してくれるようになるprint関数のwrapper
def datePrint(*args, **kwargs):
    from datetime import datetime
    print(datetime.now().strftime('[%Y/%m/%d %H:%M:%S] '), end="")
    print(*args, **kwargs)

In [3]:
datePrint("loading pickle data")
input_val0 = pickle.load(open("../data/max_span100/input_test0.pkl","rb"))
target_val0 = pickle.load(open("../data/max_span100/target_test0.pkl","rb")) #256のみ
target_val0 = torch.flip(target_val0, dims=[1])
input_train0 = pickle.load(open("../data/max_span100/input_train0.pkl","rb"))
target_train0 = pickle.load(open("../data/max_span100/target_train0.pkl","rb")) #256以下
target_train0 = torch.flip(target_train0, dims=[1])
# input_val1 = pickle.load(open("../data/max_span100/input_test0.pkl","rb"))
# target_val1 = pickle.load(open("../data/max_span100/target_test0.pkl","rb")) #256のみ
# target_val1 = torch.flip(target_val1, dims=[1])
# input_train1 = pickle.load(open("../data/max_span100/input_train0.pkl","rb"))
# target_train1 = pickle.load(open("../data/max_span100/target_train0.pkl","rb")) #256以下
# target_train1 = torch.flip(target_train1, dims=[1])

input_all = torch.cat([input_train0, input_val0], dim=0)
target_all = torch.cat([target_train0, target_val0], dim=0)
dataset = model.Dataset(input_all, target_all)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [800000, 200000])
del input_val0, target_val0, input_train0, target_train0

# input_all = torch.cat([input_train0, input_val0, input_train1, input_val1], dim=0)
# target_all = torch.cat([target_train0, target_val0, target_train1, target_val1], dim=0)
# dataset = model.Dataset(input_all, target_all)
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [1800000, 200000])

# del input_val0, target_val0, input_train0, target_train0, input_val1, target_val1, input_train1, target_train1
gc.collect()

[2021/04/07 14:57:55] loading pickle data


0

In [4]:
import math
def lambda_epoch(epoch):
    # スケジューラの設定
    max_epoch = 20
    return math.pow((1-epoch/max_epoch), 0.9)

In [ ]:
batch_size = 64
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
dataloaders_dict = {'train': train_dataloader, 'val': val_dataloader}

for n in [0]:
    for x in [0]:
        datePrint(n, 'layer', x, 'flag')
        net = model.Fixed(num_layer=32, kernel_sizes=255, flag=False).to(device)
        net.apply(model.weight_init) #重みの初期化適用

        #ファインチューニング
#         optimizer = optim.Adam([{'params': net.embedding.parameters(), 'lr': 5e-4},
#                                 {'params': net.convs.parameters(), 'lr': 1e-4},
#                                 {'params': net.mid.parameters(), 'lr': 5e-4},
#                                 {'params': net.fc.parameters(), 'lr': 1e-3}], weight_decay=1e-6)
        optimizer = optim.Adam(net.parameters(), lr=1e-4, weight_decay=1e-6, eps=1e-5)

        epochs = 20
        criterion = nn.MSELoss().to(device)
        
        
#         # 学習途中データ
#         checkpoint = torch.load('max_span100.pth')
#         net.load_state_dict(checkpoint['model_state_dict'])
#         optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#         epochs = checkpoint['epoch']
#         loss = checkpoint['loss']
        
        scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)
        train_loss_list, val_loss_list, data_all, target_all, output_all = mode.train(device, net, dataloaders_dict, criterion, optimizer, epochs)               
        torch.save(net.state_dict(), 'max_span100_variable.pth')

        result.learning_curve(train_loss_list, val_loss_list, epochs)
        result.plot_result(np.array(target_all, dtype=object).reshape(-1), np.array(output_all, dtype=object).reshape(-1))
        cor_list, loss_list = result.cal_indicators(target_all, output_all)
        result.cor_hist(cor_list)
        result.scatter_minmax(cor_list, loss_list, target_all, output_all)
        result.visible_minmax(target_all, output_all, cor_list, loss_list)
        result.show_base(data_all, cor_list, loss_list)

[2021/04/07 14:57:58] 0 layer 0 flag
Epoch 1/20
train Loss:2.8901675950050354 Timer:16091.802738666534
val Loss:4.320832475662232 Timer:1300.6681745052338
Epoch 2/20
train Loss:2.8449122081947325 Timer:16093.79679608345


In [ ]:
# input_test = pickle.load(open("../data/test_sets/input_600_1000.pkl","rb"))
# target_test = pickle.load(open("../data/test_sets/target_600_1000.pkl","rb"))
input_test = pickle.load(open("../data/RF00156/input_RF00156.pkl","rb"))
target_test = pickle.load(open("../data/RF00156/target_RF00156.pkl","rb"))
target_test = torch.flip(target_test, dims=[1])
datePrint(input_test.shape)
datePrint(target_test.shape)
if (input_test.shape[1]%256 != 0):
    input_test = F.pad(input_test, (0, 256-input_test.shape[1]%256))
    target_test = F.pad(target_test, (0, 252-target_test.shape[1]%256))
if (input_test.shape[1]>256):
    transform = True
    division = (input_test.shape[1])//128 - 1
    input_init = input_test
    input_test = input_test.unfold(1, 256, 128).reshape(-1, 256)
    target_test = target_test.unfold(1, 252, 128).reshape(-1, 252)
else:
    transform = False
datePrint(input_test.shape)
datePrint(target_test.shape)
test_dataset = model.Dataset(input_test, target_test)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64,shuffle=False, num_workers=1)

net = model.Fixed(num_layer=16, flag=False).to(device)
net.load_state_dict(torch.load('max_span100.pth'))
criterion = nn.MSELoss().to(device)

start = time.time()
test_loss, data_all, target_all, output_all = mode.test(device, net, test_dataloader, criterion) 
datePrint('finish prediction loss', test_loss)
if (transform==True):
    target_tmp = torch.tensor(target_all)
    output_tmp = torch.tensor(output_all)
    output_tmp = F.relu(output_tmp)
    for n in range(division):
        if (n==0):
            target_all = target_tmp[n::division, :192]
            output_all = output_tmp[n::division, :192]
        elif (n==division-1):
            target_all = torch.cat([target_all, target_tmp[n::division, 64:]], dim=1)
            output_all = torch.cat([output_all, output_tmp[n::division, 64:]], dim=1)
        else:
            target_all = torch.cat([target_all, target_tmp[n::division, 64:192]], dim=1)
            output_all = torch.cat([output_all, output_tmp[n::division, 64:192]], dim=1)
    
    data_all = input_init.numpy()
    target_all = target_all.numpy()
    output_all = output_all.numpy()
finish = time.time()
np.savetxt('accessibility_output.txt', output_all, fmt='%.3e')
finish = time.time()
datePrint('予測時間', (finish-start))

In [ ]:
datePrint('test_loss: {:.3f}'.format(test_loss))
result.plot_result(np.array(target_all).reshape(-1), np.array(output_all).reshape(-1))
cor_list, loss_list = result.cal_indicators(target_all, output_all)
result.cor_hist(cor_list)
result.scatter_minmax(cor_list, loss_list, target_all, output_all)

In [ ]:
result.visible_minmax(target_all, output_all, cor_list, loss_list)
result.show_base(data_all, cor_list, loss_list)

In [ ]:
for i in range(4):
    result.visible_one(target_all, output_all)

In [ ]:
start = time.time()
path = "../data/mouse_data/seq3.fa"
with open(path, mode = 'r', encoding = 'utf-8') as f:
    seq_file = f.read().splitlines()
indexes = [i for i, n in enumerate(seq_file) if n.startswith('>')]
name = seq_file[0]
seq = ''.join(seq_file[1:]).replace('A', '1').replace('T', '2').replace('U', '2').replace('G', '3').replace('C', '4')
input_seq = torch.Tensor(list(map(int, seq)))
input_seq = torch.flip(input_seq, dims=[0])
input_seq = input_seq.unsqueeze(0)
out_length = len(input_seq[0])-4

if (input_seq.shape[1]%256 != 0):
    input_seq = F.pad(input_seq, (0, 256-input_seq.shape[1]%256))
if (input_seq.shape[1]>256):
    transform = True
    division = (input_seq.shape[1])//128 - 1
    input_init = input_seq
    input_seq = input_seq.unfold(1, 256, 128).reshape(-1, 256)
else:
    transform = False
input_seq = input_seq.unsqueeze(0)

net = model.Fixed(num_layer=16, flag=False).to(device)
net.load_state_dict(torch.load('256_finish.pth'))
    
data_all, output_all = mode.predict(device, net, input_seq) 

if (transform==True):
    output_tmp = torch.tensor(output_all)
    for n in range(division):
        if (n==0):
            output_all = output_tmp[n::division, :192]
        elif (n==division-1):
            output_all = torch.cat([output_all, output_tmp[n::division, 64:]], dim=1)
        else:
            output_all = torch.cat([output_all, output_tmp[n::division, 64:192]], dim=1)
    
    data_all = input_init.numpy()
#     output_all = torch.flip(output_all, dims=[1])
    output_all = output_all.numpy()
  
import matplotlib.pyplot as plt
max_length = output_all.shape[1]
with open('../data/mouse_data/out3.txt', 'r') as f:
        next(f)
        acc = f.readlines()
        acc_list = []
        for i in range(len(acc)-1):
                acc_list.append(re.findall(',(.*);', acc[i])[0])
acc_list = [float(x) for x in acc_list]

plt.figure(figsize=(15, 7))
plt.plot(range(out_length), acc_list, label='target', color='b')
plt.plot(range(out_length), output_all[0][:out_length], label='output', color='r')
plt.legend()
plt.xlabel('base position')
plt.ylabel('accessibility')
plt.title('one')
plt.show()


cor = np.corrcoef(acc_list, output_all[0][:out_length])
mse = ((acc_list - output_all[0][:out_length])**2).mean(axis=0)
print('cor', cor[0,1])
print('mse', mse)

    
np.savetxt('accessibility_output.txt', output_all, fmt='%.3f')
finish = time.time()
datePrint('予測時間', (finish-start))

In [ ]:
path = "../data/real_data/RF00156.fa"

start = time.time()
with open(path, mode = 'r', encoding = 'utf-8') as f:
    seq_file = f.read().splitlines()
indexes = [i for i, n in enumerate(seq_file) if n.startswith('>')]
name_list = []
seq_list = torch.empty(0, 256)

for i in range(len(indexes)-1):
    name_list.append(seq_file[indexes[i]])
    seq = ''.join(seq_file[indexes[i]+1:indexes[i+1]]).replace('A', '1').replace('T', '2').replace('U', '2').replace('G', '3').replace('C', '4').replace('N', '0')
    seq = torch.Tensor(list(map(int, seq)))
    seq = torch.flip(seq, dims=[0])
    if (seq.shape[0]%256 != 0):
        seq = F.pad(seq, (0, 256-seq.shape[0]%256))
    seq = seq.unsqueeze(0)
    seq_list = torch.cat([seq_list, seq], dim=0)


# if (input_seq.shape[1]%256 != 0):
#     input_seq = F.pad(input_seq, (0, 256-input_seq.shape[1]%256))
# if (input_seq.shape[1]>256):
#     transform = True
#     division = (input_seq.shape[1])//128 - 1
#     input_init = input_seq
#     input_seq = input_seq.unfold(1, 256, 128).reshape(-1, 256)
# else:
#     transform = False
# datePrint(input_seq.shape)
# input_seq = input_seq.unsqueeze(0)
# datePrint(input_seq.shape)

seq_list = seq_list.unsqueeze(0)
net = model.Fixed(num_layer=16, flag=False).to(device)
net.load_state_dict(torch.load('256_finish.pth'))
    
data_all, output_all = mode.predict(device, net, seq_list) 
# if (transform==True):
#     output_tmp = torch.tensor(output_all)
#     for n in range(division):
#         if (n==0):
#             output_all = output_tmp[n::division, :192]
#         elif (n==division-1):
#             output_all = torch.cat([output_all, output_tmp[n::division, 64:]], dim=1)
#         else:
#             output_all = torch.cat([output_all, output_tmp[n::division, 64:192]], dim=1)
    
#     data_all = input_init.numpy()
#     output_all = output_all.numpy()
    
np.savetxt('accessibility_output.txt', output_all, fmt='%.3f')
finish = time.time()
datePrint('予測時間', (finish-start))

In [ ]:
# sample作成
path = "../data/RF01210/RF01210.fa"

with open(path, mode = 'r', encoding = 'utf-8') as f:
    seq_file = f.read().splitlines()
indexes = [i for i, n in enumerate(seq_file) if n.startswith('>')]

for i in range(len(indexes)-1):
    with open(f"../data/RF01210/sample_{i}.txt", mode="w") as f:
        f.write(seq_file[indexes[i]] + "\n")
        f.write(''.join(seq_file[indexes[i]+1:indexes[i+1]]))

In [ ]:
# castしちゃう
from pathlib import Path
import numpy as np
from tqdm import tqdm
import pickle
import torch
import sys

for cond, cnt in (("train", 500000), ("test", 500000)):
    data_path = Path(f"../data/makedata/{cond}")
    input_array = []
    target_array = []
    print(f"reading {cond} files")
    for i in tqdm(range(cnt)):
        input_path = data_path / f"index/input_{i+1}.csv"
        target_path = data_path / f"accessibility/target_{i+1}.csv"

        input_array.append(torch.Tensor(np.loadtxt(input_path, delimiter=",", dtype=np.float).astype(np.int)))
        target_array.append(torch.Tensor(np.loadtxt(target_path, delimiter=",", dtype=np.float)))
    print(f"saving to input_{cond}.pkl")
    pickle.dump(torch.stack(input_array), open(f"../data/input_{cond}.pkl", 'wb'))
        
    print(f"saving to target_{cond}.pkl")
    pickle.dump(torch.stack(target_array), open(f"../data/target_{cond}.pkl", 'wb'))


In [ ]:
#メモリ確認
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 10000: #10M以上のみ表示
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))